Notebook to read and fix hfout output times

This script leverages the CDO (https://code.mpimet.mpg.de/projects/cdo/) module via subprocess (executes as a terminal command) to generate basic time series, which helps for checking for RCE. This should be available either by loading as a module or installing into the conda/mamba kernel you're running.

James Ruppert  
21 April 2024

In [1]:
import subprocess
from read_wrf_ideal import *

## Main Settings

In [2]:
# Small domain model time specs
npd_wrf = 48 # time steps per day
npd_hf = 288 # time steps per day
# npfile = npd # time steps per output file
nd=50 # total n-days of run

# Vertical interpolation settings
dp_int = 20 # hPa
top_p = 100

### Directories and model output files

In [3]:
test_process='ctl'
# test_process='xcrf'

# wrfdir = "/glade/derecho/scratch/ruppert/wrf-ideal/largedom2/"
wrfdir = "/glade/campaign/univ/uokl0049/ideal/largedom2/"+test_process+"/"
workdir = "/glade/work/ruppert/wrf-ideal/"
wrftag = "wrfout_d01"
hftag = "hfout_d01"

figdir = "/glade/u/home/ruppert/figures/wrf-ideal/"

# WRFOUT files
wrffiles = get_wrf_file_list(wrfdir, wrftag)
# times_wrf = get_times_wrffiles(wrffiles)
# print('WRF file list contains',times_wrf.size,'time steps')
# print('WRF file list contains',times_wrf.size/npd_wrf,'days')

# High-frequency (HFOUT) files
hffiles = get_wrf_file_list(wrfdir, hftag)
# print('HF file list contains',times_hf.size-1,'time steps')
# print('HF file list contains',(times_hf.size-1)/npd_hf,'days')

## Fix output time steps

At model restarts, output from special streams begins at output_time_step + 1, yielding inconsistent files from then onward, e.g., starting from 00:05 UTC instead of 00:00 UTC.

The problem is corrected simply: this code block creates a copy of the first time step of the pre-restart output file, with the original naming convention. The original file is first saved as *sav.hfout...*). The new file will then have a single time step but will be in the correct sequence for future *cdo mergetime* calls.

In [11]:
# do_fix_output_times = True
do_fix_output_times = False

# Create string array of file-specific commands
if do_fix_output_times:
    # for ifil in range(len(hffiles)):
    for ifil in range(23,24):
        if hffiles[ifil][-4] == '5':
            print(hffiles[ifil])
            # First save a copy of the original, then 
            outfname = 'sav.'+hffiles[ifil-1].split('/')[-1]
            outfile = wrfdir+outfname
            process = subprocess.Popen(['mv '+hffiles[ifil-1]+' '+outfile], shell=True, stdout=subprocess.PIPE, universal_newlines=True)
            lines = process.stdout.readlines()
            for iline in lines:
                print(iline)
            print()
            process = subprocess.Popen(['cdo copy -seltimestep,1 '+outfile+' '+hffiles[ifil-1]], shell=True, stdout=subprocess.PIPE, universal_newlines=True)
            lines = process.stdout.readlines()
            for iline in lines:
                print(iline)
            continue